In [ ]:
# Install necessary libraries
# (Run these in your shell before executing the script, not within the script)
!pip install gradio -q
!python -m pip install floret 'spacy~=3.6.0' --quiet
!wget -nc https://github.com/explosion/spacy-vectors-builder/releases/download/en-3.4.0/en_vectors_floret_md.floret.gz
!spacy init vectors en en_vectors_floret_md.floret.gz en_vectors_floret_md --mode floret


In [ ]:

# Imports
import io
import pandas as pd
import numpy as np
import spacy
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# spaCy pipeline with floret vectors
nlp_fl = spacy.load("en_vectors_floret_md")

In [ ]:
# Data can also be in a file or database externally

data_txt = """

{"question": "When should staff members report sick for sickness absence?", "answer": "Staff members must report sick when working hours begin on their first day of sickness absence or at the time of day they leave their workplace due to sickness."}
{"question": "What should be done if a staff member falls ill during the workday and leaves the workplace?", "answer": "If a staff member falls ill during the workday and leaves the workplace, the remaining part of the day must be registered as sickness absence."}
{"question": "What should be done when a staff member reports sick when working hours begin?", "answer": "When a staff member reports sick when working hours begin, the entire workday must be registered as sickness absence."}
{"question": "What information should staff members provide when reporting absence from work due to medical examinations and treatments?", "answer": "Staff members must provide information on the dates of their sickness absence when possible."}
{"question": "When should a manager invite an absent staff member for a meeting?", "answer": "A manager must invite the absent staff member for a meeting after 5, 14, and 28 consecutive days of sickness absence."}
{"question": "What is the purpose of the meeting between a manager and a staff member in case of sickness absence?", "answer": "The meeting is held at an early stage to help prevent long periods of sickness absence and ensure that staff members are aware of their future options."}
{"question": "Where can I find additional information on regulatory obligations and possibilities regarding long-term sickness absence?", "answer": "More detailed information on regulatory obligations and possibilities is available in the AAU Handbook."}
{"question": "What materials/tools are available on the HR Department's website for handling sickness absence?", "answer": "Various materials/tools, such as guides for sickness absence review meetings and templates for action plans and evaluation and retention, are available on the HR Department's website."}
{"question": "Who can I contact for questions about sickness absence management?", "answer": "You can contact the HR Department at Hr-fravaer@adm.aau.dk for questions about sickness absence management."}
{"question": "What is carer's leave, and who is entitled to it?", "answer": "Carer's leave is self-paid leave for staff members to provide essential care or support to a close family member with a serious health condition. All staff members are entitled to it."}
{"question": "Is there a transition period for carer's leave entitlement?", "answer": "Yes, there is a transition period from 2 August to 31 December 2022, during which staff members have the right to take two working days of carer's leave."}
{"question": "Who qualifies as a close family member for carer's leave purposes?", "answer": "Close family members include one's own children, parents, spouse or partner, or a person living in the same household as the staff member."}
{"question": "In what situations can carer's leave be used?", "answer": "Carer's leave can be used when providing personal care or support in specific situations, such as accompanying a relative to a medical examination or providing care to a sick person in various settings."}
{"question": "Is medical documentation required for carer's leave?", "answer": "The employer may require medical documentation in some cases. It is up to the employer to decide whether it is necessary."}
{"question": "How should I report taking carer's leave?", "answer": "To report carer's leave, contact the department/institute via email or telephone and send a confirmation via email on the specific day(s) you are taking carer's leave."}
{"question": "What does the accident insurance for students at Aalborg University cover?", "answer": "The accident insurance covers injuries sustained by students during laboratory activities, including experiments, fieldtrips, excursions, and internships as part of their studies."}
{"question": "What types of injuries are covered by the accident insurance?", "answer": "The accident insurance allows students to claim compensation for medical expenses, rehabilitation expenses, compensation for permanent injury, and dental injury directly caused by laboratory activities."}
{"question": "How can I report an injury covered by the accident insurance?", "answer": "You should immediately fill in the accident report form, available in both English and Danish, and submit it to sts-sekretariatet@adm.aau.dk, which will forward it to the insurance broker for further processing."}
{"question": "Does the accident insurance cover all AAU students?", "answer": "The accident insurance covers all AAU students who participate in laboratory activities as part of their studies, except for PhD students employed by AAU."}
{"question": "Does the accident insurance include liability insurance?", "answer": "No, the accident insurance does not include liability insurance. Students are advised to take out a personal accident and liability insurance."}
{"question": "Who provides the accident insurance for AAU students?", "answer": "The accident insurance is provided by IF Skadeforsikring, with policy number SP1526844, and is valid as of 1 January 2018."}
{"question": "Whom can I contact for questions about completing the accident report form?", "answer": "You can contact Jimmy Leonhardt Schleisner at jimmy.schleisner@willistowerswatson.com or by phone at +45 3148 0648 for assistance with the accident report form."}
{"question": "For other questions related to the accident insurance, whom should I contact?", "answer": "You can contact Sine Sø Kristensen at sts-sekretariatet@adm.aau.dk or by phone at +45 9940 7618 for other questions related to the accident insurance."}
{"question": "What are the rules for admission and enrolment at Aalborg University?", "answer": "The rules for admission and enrolment at Aalborg University are outlined in the document published on 31.01.2018 and last revised on 18.02.2021."}
{"question": "Which study programs do the rules apply to?", "answer": "The rules apply to applications for admission to the first semester of a bachelor program, a professional bachelor program, a master program, including the four-year master program, at Aalborg University."}
{"question": "Where can I find the admission requirements for bachelor programs and professional bachelor programs?", "answer": "The admission requirements for bachelor programs and professional bachelor programs can be found in the ministerial orders on admission and enrolment on bachelor programs and professional bachelor programs."}
{"question": "How should I apply for admission to the first year of a bachelor program or professional bachelor program?", "answer": "Applications for admission to the first year (first and second semesters) of a bachelor program and professional bachelor program must be submitted through the Coordinated Admission System."}
{"question": "Under what conditions can applicants be enrolled on the third semester or any subsequent semester of a bachelor program or professional bachelor program without applying through the Coordinated Admission System?", "answer": "Applicants who are currently enrolled at a full-time program at Aalborg University, another Danish university or university college, and who have completed or may receive credit transfer for the program elements of the entire first year of the study program in question, as well as applicants who have previously been enrolled at the same program and who have completed or may receive credit transfer for the program elements of the entire first year of the study program in question, can be enrolled without applying through the Coordinated Admission System."}
{"question": "What are the requirements for admission to master programs at Aalborg University?", "answer": "In order to be admitted to master programs, applicants must meet the admission requirements stipulated in chapter 3 of the Danish Ministerial Order on Admission to Full-Time University Programs, meet the language requirements stipulated by the University, and ensure that study places are available on the specific semester of the program on which the applicant is applying for admission or enrolment."}
{"question": "Do applicants holding a bachelor's degree from Aalborg University have any special admission rights to master programs?", "answer": "Applicants holding a bachelor's degree from Aalborg University have a legal claim for admission to the master program that constitutes the natural extension of the subject area of their bachelor program, provided that the applicant is admitted no later than three years from the completion of the bachelor program. Applicants who have completed a master program or have completed their bachelor program before 1 January 2019 do not have legal claim for admission."}
{"question": "What are the additional requirements for admission to the four-year master program?", "answer": "In addition to the academic admission requirements, applicants must either have relevant employment with a public or private employer obtained on the basis of their relevant bachelor's degree or have their own relevant entrepreneurial business. The workload must equal on average at least 25 hours a week. Under special circumstances, the university can grant an exemption from the minimum number of weekly working hours, provided that the employment meets the relevance requirement."}
{"question": "What is the application deadline for admissions to a bachelor program or professional bachelor program submitted through the Coordinated Admission System?", "answer": "The application deadline for admissions to a bachelor program or professional bachelor program submitted through the Coordinated Admission System is specified in the ministerial order on admission and enrolment on bachelor programs and the ministerial order on admission and enrolment on academy profession programs and professional bachelor programs."}
{"question": "How many master programs can applicants apply for at Aalborg University?", "answer": "Applicants may apply for admission to up to three master programs at Aalborg University in order of priority."}
{"question": "How are complaints regarding decisions made on the basis of these regulations handled?", "answer": "Complaints regarding decisions made on the basis of these regulations may be submitted to the Danish Agency for Science and Higher Education, provided the complaint concerns legal questions. The complaint must be submitted to the Admissions Office no later than two weeks after the complainant has been notified of the decision."}
{"question": "When do these rules take effect?", "answer": "These rules took effect on 12 February 2021 and include all students enrolled on this date, persons applying for admission or enrolment, and all students admitted or enrolled after this date."}

"""

In [ ]:
# Load data into DataFrame
buffer = io.StringIO(data_txt)
df = pd.read_json(buffer, lines=True)

In [ ]:
# Obtain vectors for the questions in the dataset
vecs = np.vstack([txt.vector for txt in nlp_fl.pipe(df.question, disable=["parser", "ner"])])


In [ ]:
# Name for the bot
bot_name = '🌸 Floret Mads'

def reply_fl(txt):
    """
    Generate a reply based on the input text by finding the most similar
    questions in the existing data, and responding accordingly.
    """
    # Find cosine similarity between input text vector and all existing question vectors
    sims = cosine_similarity(vecs, nlp_fl(txt).vector.reshape(1, -1))
    sorted_sims = np.sort(sims, axis=0)[::-1]
    sorted_indices = np.argsort(sims, axis=0)[::-1]

    # If highest similarity is above threshold, return corresponding answer
    if sorted_sims[0][0] >= 0.95:
        print(f"{bot_name}: {df.iloc[sorted_indices[0][0],1]}")
    else:
        # Suggest the most similar questions if similarity is below threshold
        print(f"{bot_name}: Did you mean one of these questions?: \n {df.iloc[sorted_indices[0][0],0]} \n {df.iloc[sorted_indices[1][0],0]} \n Please enter the question again 🙏💪")


In [ ]:

# Example query
reply_fl("Who qualifies as a close family member for carer's leave purposes")